In [1]:
!sudo apt-get install git-lfs

!git lfs install

!git clone https://****************@huggingface.co/jc-cl-cg/llm_anthro_detection

!cd llm_anthro_detection
!git config --global user.email "***********"

!git config --global user.name "***********"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
fatal: destination path 'llm_anthro_detection' already exists and is not an empty directory.


# Install comet_ml and create experiment

In [3]:
%pip install comet_ml torch datasets transformers scikit-learn
import comet_ml

experiment = comet_ml.Experiment(
    api_key="***********",
     project_name="LLM-anthro", log_code=True,
    auto_metric_logging=True,
    auto_param_logging=True,
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/cngish98/llm-anthro/ef603d9ce0b3451192382353cac864f8



# log params for use later

In [4]:
# class weights for unbalanced classes
weight_for_0 = (1 / 6610) * (7380 / 2.0)
weight_for_1 = (1 / 770) * (7380 / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}


params = {
    "bert": "bert-base-uncased",
    "num_labels": 2,
    "return_tensors": "tf",
    "batch_size": 32,            
    "epochs": 10,                  # increase num of epochs
    "padding":"max_length",
    "truncation": True,
    "class_weight": class_weight
}

experiment.log_parameters(params)

# tokenization

In [5]:
def tokenize_function(examples):
    from transformers import BertTokenizer
    tokenizer = BertTokenizer.from_pretrained(params['bert'])
    return tokenizer(examples["text"], padding=params["padding"], truncation=params["truncation"])


# create datasets for training and evaluation

In [6]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'train': '/content/gold_train.csv',
                                              'test': '/content/gold_test.csv'})
#print(dataset)

#tokenizer = AutoTokenizer.from_pretrained(params['bert'])
tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)  # .select(range(1000))

print(small_train_dataset)
print(small_eval_dataset)


  0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/5904 [00:00<?, ? examples/s]

Map:   0%|          | 0/1476 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5904
})
Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1476
})


In [7]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors=params['return_tensors'])

tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=params['batch_size'],)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=params['batch_size'],)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [8]:
import tensorflow as tf
from tensorflow import keras
from transformers import TFAutoModelForSequenceClassification
%pip install tensorflow-addons
import tensorflow_addons as tfa
from keras.losses import binary_crossentropy, categorical_crossentropy
#%pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [10]:



bert = TFAutoModelForSequenceClassification.from_pretrained(params['bert'], num_labels=params['num_labels'])


bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=tf.metrics.SparseCategoricalAccuracy('accuracy'))


bert.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=params['epochs'], class_weight=class_weight)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
COMET WARNING: tensorflow datasets are not currently supported for gradient and activation auto-logging


Epoch 1/10
185/185 [==============================] - 156s 592ms/step - loss: 0.5241 - accuracy: 0.7116 - val_loss: 0.4577 - val_accuracy: 0.7473
Epoch 2/10
185/185 [==============================] - 103s 555ms/step - loss: 0.3204 - accuracy: 0.8613 - val_loss: 0.5522 - val_accuracy: 0.7798
Epoch 3/10
185/185 [==============================] - 101s 544ms/step - loss: 0.1735 - accuracy: 0.9287 - val_loss: 0.4255 - val_accuracy: 0.8537
Epoch 4/10
185/185 [==============================] - 100s 542ms/step - loss: 0.1240 - accuracy: 0.9466 - val_loss: 0.4214 - val_accuracy: 0.8713
Epoch 5/10
185/185 [==============================] - 99s 538ms/step - loss: 0.0838 - accuracy: 0.9632 - val_loss: 0.6464 - val_accuracy: 0.8178
Epoch 6/10
185/185 [==============================] - 100s 540ms/step - loss: 0.0574 - accuracy: 0.9797 - val_loss: 0.4107 - val_accuracy: 0.9058
Epoch 7/10
185/185 [==============================] - 99s 537ms/step - loss: 0.0371 - accuracy: 0.9864 - val_loss: 0.5037 - v

In [12]:
bert.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [13]:
from transformers import BertTokenizer
input_list = ['GPT-3 was used to create this piece of art.',
              'The model generated a long response.',
              'He used ChatGPT to summarize a topic.',
              'ChatGPT comforts me when I am upset.'] 
for sentence in input_list:
  experiment.log_text(sentence)
  # encode context the generation is conditioned on
  tokenizer = BertTokenizer.from_pretrained(params['bert'])
  input_ids = tokenizer.encode(sentence, return_tensors='tf')
  output = bert(input_ids)
  logits = output.logits

  predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
  prediction = bert.config.id2label[predicted_class_id]
  experiment.log_text(prediction)
  print(sentence, prediction)

GPT-3 was used to create this piece of art. LABEL_0
The model generated a long response. LABEL_0
He used ChatGPT to summarize a topic. LABEL_0
ChatGPT comforts me when I am upset. LABEL_1


In [14]:
bert.save_pretrained('/content/llm_anthro_detection')

In [17]:
cd llm_anthro_detection

/content/llm_anthro_detection


In [18]:
!git add --all
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   config.json
	new file:   tf_model.h5



In [19]:
!git commit -m "first version of llm_anthro_detection model"

[main 1829f96] first version of llm_anthro_detection model
 2 files changed, 28 insertions(+)
 create mode 100644 config.json
 create mode 100644 tf_model.h5


In [20]:
!git push

Uploading LFS objects: 100% (1/1), 438 MB | 13 MB/s, done.
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 12 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 836 bytes | 836.00 KiB/s, done.
Total 4 (delta 0), reused 0 (delta 0)
To https://huggingface.co/jc-cl-cg/llm_anthro_detection
   33267bd..1829f96  main -> main


In [21]:
tokenizer.save_pretrained('/content/llm_anthro_detection')

('/content/llm_anthro_detection/tokenizer_config.json',
 '/content/llm_anthro_detection/special_tokens_map.json',
 '/content/llm_anthro_detection/vocab.txt',
 '/content/llm_anthro_detection/added_tokens.json')

In [22]:
!git add --all
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   special_tokens_map.json
	new file:   tokenizer_config.json
	new file:   vocab.txt



In [23]:
!git commit -m "add tokenizer to first version of model"

[main f7777e9] add tokenizer to first version of model
 3 files changed, 30544 insertions(+)
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer_config.json
 create mode 100644 vocab.txt


In [24]:
!git push

Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 12 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 108.83 KiB | 4.35 MiB/s, done.
Total 5 (delta 0), reused 0 (delta 0)
To https://huggingface.co/jc-cl-cg/llm_anthro_detection
   1829f96..f7777e9  main -> main


In [25]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/cngish98/llm-anthro/ef603d9ce0b3451192382353cac864f8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.7115514874458313, 0.9894986152648926)
COMET INFO:     batch_accuracy [190]         : (0.40625, 1.0)
COMET INFO:     batch_loss [190]             : (0.002561744535341859, 0.6710404753684998)
COMET INFO:     epoch_duration [10]          : (85.21205011200027, 156.52704476600002)
COMET INFO:     loss [10]                    : (0.034868232905864716, 0.5241197347640991)
COMET INFO:     val_accuracy [10]            : (0.7472899556159973, 0.9065040946006775)
COMET INFO:     val_loss [10]       